# Examples on how ADFlib works

In [1]:
from ADFlib import *
%load_ext autoreload
%autoreload 2

## Base level factors

In [2]:
# A base level factor contains an id, text and truth value 
blf = BaseLevelFactor(0, 'Did the crime occur?', True)
blf

0: Did the crime occur?[True]

In [3]:
blf2 = BaseLevelFactor(1, 'Was there a victim?', False)
blf2

1: Was there a victim?[False]

## Relationships

In [4]:
# OR relations take 2 or more base level factors or relations
r_OR = Relationship('OR', [blf, blf2])
print(r_OR, '\n> Evaluates to', r_OR.evaluate())

   0: Did the crime occur?[True] OR
   1: Was there a victim?[False] 
> Evaluates to True


In [5]:
# NOT relations take 1 base level factors
r_NOT = Relationship('NOT', blf2)
print(r_NOT, '\n> Evaluates to',  r_NOT.evaluate())

NOT 1: Was there a victim?[False] 
> Evaluates to True


In [6]:
# AND relations take 2 or more base level factors or relations
r_AND = Relationship('AND', [blf, r_NOT])
print(r_AND, '\n> Evaluates to',  r_AND.evaluate())

   0: Did the crime occur?[True] AND
   NOT 1: Was there a victim?[False] 
> Evaluates to True


## Abstract factors

In [7]:
# abstract factors contain an identifier, text, default evaluation, 
# the factors for acceptance and factors for rejection (both of which 
# are defined in terms of a relationship)
af = AbstractFactor(identifier=2, 
                    text='This is an abstract factor', 
                    default=False,
                    acceptfactors=r_AND,
                    rejectfactors=None)
print(af, '\n> Evaluates to',  af.evaluate())

2: This is an abstract factor[True]
   ACCEPT 2 IF:
   0: Did the crime occur?[True] AND
   NOT 1: Was there a victim?[False]
   REJECT otherwise
 
> Evaluates to True


## ADFs

In [8]:
# an ADF is a subclass of an abstract factor with some additional functions
adf = ADF(identifier=2, 
          text='This is an ADF', 
          default=False,
          acceptfactors=r_OR,
          rejectfactors=r_NOT)
print(adf, '\n> Evaluates to',  adf.evaluate())

2: This is an ADF[False]
   REJECT 2 IF:
NOT 1: Was there a victim?[False]
   ACCEPT 2 IF:
   0: Did the crime occur?[True] OR
   1: Was there a victim?[False]
   REJECT otherwise
 
> Evaluates to False


In [9]:
# List all of the baselevel factors
adf.get_baselevel_factors()

[0: Did the crime occur?[True], 1: Was there a victim?[False]]

In [10]:
# Change the values of the baselevel factors of the ADF
vals = {0:False} # Changing the value of base level factor 0 to False
adf.set_baselevel_factors(vals)
adf.get_baselevel_factors()

[0: Did the crime occur?[False], 1: Was there a victim?[False]]

# Example: Article 6 of the European Convention on Human Rights

In [11]:
# Import the ADF from article6.py
from article6 import art6_ADF

# Show the ADF
art6_ADF

violation: Violation of Article 6?[False]
   ACCEPT violation IF:
   I1: Is the applicant a victim?[False] AND
   I2: The applicant was admissible[False]
      ACCEPT I2 IF:
         I2Q1: Is the case well founded?[False] AND
         I2F1: The victim suffered a disadvantage.[False]
         ACCEPT I2F1 IF:
               I2F1Q1: The case examines a fundamental aspect?[False] AND
               I2F1Q2: Have all Domestic courts have been exhausted?[False]
         REJECT otherwise
      
      REJECT otherwise
    AND
   I3: Fair and Public.[False]
      ACCEPT I3 IF:
         I3F1: Conducted in Reasonable Time.[False]
         ACCEPT I3F1 IF:
               I3F1Q2: Did the government cause any unreasonable delays?[False] AND
               I3F1Q1: Was the case conducted in a reasonable time?[False]
         REJECT otherwise
       AND
         I3F2: The case was independent and impartial.[False]
         ACCEPT I3F2 IF:
               I3F2Q1: The government was subjectively impartial?[

# Creating data from the ADF

In [12]:
df = art6_ADF.create_dataset(1000)
df.to_csv('datasets/article6.csv')
df

,I1,I2Q1,I2F1Q1,I2F1Q2,I3F1Q2,I3F1Q1,I3F2Q1,I3F2Q2,I3F3Q1,I3F3Q2,...,I5Q1,I5F2Q3,I5F2Q2,I5F2Q1,I5F3Q1,I5F3Q2,I5F4Q1,I5F4Q2,I5Q2,violation
0,1,1,1,1,1,1,0,1,1,0,...,1,1,1,1,1,1,1,1,0,False
1,1,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,1,1,0,1,False
2,1,0,1,1,0,1,0,0,0,0,...,0,0,0,1,1,1,1,0,1,False
3,1,0,1,0,1,1,0,0,1,1,...,0,1,0,0,0,1,0,1,1,False
4,1,0,1,0,0,0,1,1,1,0,...,1,0,1,0,0,1,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,1,0,1,0,1,1,0,1,...,0,1,0,1,1,1,1,1,1,False
996,1,0,0,1,0,0,1,1,0,1,...,0,1,1,1,0,1,1,0,1,False
997,1,1,0,0,0,0,0,1,0,0,...,1,1,0,1,0,1,1,0,1,False
998,1,1,0,1,0,0,0,1,1,0,...,1,0,0,0,1,1,0,0,0,False


### Explaining the outcome of the ADF 

In [13]:
print(art6_ADF.explain())

We examined the question: 'Violation of Article 6?' and determine that this is False.
We believe this to be the case, as we can answer yes to the following questions:
	Is the applicant a victim?
	Is the case well founded?
	Was the case conducted in a reasonable time?
	The government was subjectively impartial?
	If the case was public, the public wouldn’t prejudice the outcome?
	The safety of the public wouldn’t be impacted, if the case was public?
	Was the case conducted publicly?
	Was the victim given appropriate access to Court?
	Can the highest court be considered binding in its findings?
	Was the case was reopened due to new facts or a fundamental defect in fairness?
	Are there profound and long-standing differences in the case law?
	Have tools have been used to overcome any difference in case law?
	Any doubts benefited applicant?
	Was the applicant given details of the case?
	Witnesses were examined under same conditions?
	Witnesses had a valid reason for non attendance?
	The appl